In [86]:
# Paths
DATA_PATH = "/Users/andrewrodriguez/Desktop/compsci1050/anonymity_and_ethics/reduced_qi_filled.csv"
OUT_RECORD_SUPP = "/Users/andrewrodriguez/Desktop/compsci1050/anonymity_and_ethics/de-identified_data/record_suppression.csv"
OUT_COLUMN_SUPP = "/Users/andrewrodriguez/Desktop/compsci1050/anonymity_and_ethics/de-identified_data/column_suppression.csv"
OUT_GEN = "/Users/andrewrodriguez/Desktop/compsci1050/anonymity_and_ethics/de-identified_data/generalized.csv"
OUT_COMBINATION = "/Users/andrewrodriguez/Desktop/compsci1050/anonymity_and_ethics/de-identified_data/combination.csv"


In [11]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import re
from typing import List

# Read data
df = pd.read_csv(DATA_PATH)
print("Rows:", len(df))
print("Columns:", list(df.columns))
df.head(3)
print(df.shape)

Rows: 199999
Columns: ['course_id', 'user_id', 'cc_by_ip', 'city', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events']
(199999, 15)


In [3]:
# so user_id and course_id are idenitifiers here
identifiers = ["user_id", "course_id"]
qi_cols = [c for c in df.columns if c not in identifiers]

print("Identifiers:", identifiers)
print("QI columns ({}):".format(len(qi_cols)), qi_cols)

Identifiers: ['user_id', 'course_id']
QI columns (13): ['cc_by_ip', 'city', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events']


In [4]:
# Okay lets check our baseline k anon
# to do so im gonna make a helper
from typing import List, Tuple

def k_anonymity_level(data: pd.DataFrame, qis: List[str]) -> int:
    # So the level of k anonmyity is data grouped by quasi ids
    return int(data.groupby(qis, dropna=False).size().min())

k0 = k_anonymity_level(df, qi_cols)
print("Baseline k-anonymity:", k0)

Baseline k-anonymity: 1


In [6]:
def record_suppression_k(data: pd.DataFrame, qis: List[str], k: int = 5) -> Tuple[pd.DataFrame, int]:
    # Okay so for record supression we only keep rows of size k
    # where k is 5
    sizes = data.groupby(qis, dropna=False).size().rename("size")
    tmp = data.join(sizes, on=qis)
    kept = tmp[tmp["size"] >= k].drop(columns=["size"])
    deleted = len(tmp) - len(kept)
    return kept, deleted


In [7]:
# Now we want to record supress to k = 5
rs_df, rs_deleted = record_suppression_k(df, qi_cols, k=5)
rs_k = k_anonymity_level(rs_df, qi_cols)
rs_df.to_csv(OUT_RECORD_SUPP, index=False)

print({
    "rows_source": len(df),
    "deleted_rows": rs_deleted,
    "rows_kept": len(rs_df),
    "k_after": rs_k,
    "file": OUT_RECORD_SUPP
})

{'rows_source': 199999, 'deleted_rows': 150286, 'rows_kept': 49713, 'k_after': 5, 'file': '/Users/andrewrodriguez/Desktop/compsci1050/anonymity_and_ethics/de-identified_data/record_suppression.csv'}


In [9]:
# this is the check for the column supression I mentioned in the report
for q in qi_cols:
    print("with just column", q, "k =", k_anonymity_level(df, q))

with just column cc_by_ip k = 1
with just column city k = 1
with just column postalCode k = 1
with just column LoE k = 535
with just column YoB k = 1
with just column gender k = 978
with just column nforum_posts k = 1
with just column nforum_votes k = 1
with just column nforum_endorsed k = 1
with just column nforum_threads k = 1
with just column nforum_comments k = 1
with just column nforum_pinned k = 1
with just column nforum_events k = 1


In [8]:
def greedy_column_suppression_until_k(data, identifiers, target_k=5, count_na_as_value=True, verbose=False):
    # Start with all QIs
    current_qis = [c for c in data.columns if c not in identifiers]
    dropped = []

    # Current k
    current_k = k_anonymity_level(data, current_qis)
    if verbose:
        print(f"Start: k={current_k}, QIs={len(current_qis)}")

    # How to count uniques (count NaN as its own value or not)
    nunique_kwargs = {"dropna": not count_na_as_value}

    # Precompute per-column cardinalities (they don’t change as we drop other columns)
    cardinality = {c: data[c].nunique(**nunique_kwargs) for c in current_qis}

    while current_k < target_k and current_qis:
        # Find the maximum cardinality among remaining QIs
        max_card = max(cardinality[c] for c in current_qis)
        candidates = [c for c in current_qis if cardinality[c] == max_card]

        # Tie-break among candidates: pick the one whose removal yields the highest k,
        # then the fewest groups, then lexicographically.
        best_col = None
        best_k = -1
        best_groups = None
        best_name = None

        for c in candidates:
            trial_qis = [x for x in current_qis if x != c]
            kval = k_anonymity_level(data, trial_qis)
            ng = (data.groupby(trial_qis, dropna=False).ngroups if trial_qis else 1)

            if (kval > best_k) or \
               (kval == best_k and (best_groups is None or ng < best_groups)) or \
               (kval == best_k and ng == best_groups and (best_name is None or c < best_name)):
                best_k = kval
                best_groups = ng
                best_col = c
                best_name = c

        # Drop the chosen column
        current_qis.remove(best_col)
        dropped.append(best_col)
        current_k = best_k
        cardinality.pop(best_col, None)

        if verbose:
            print(f"Dropped '{best_col}' (card={max_card}) -> k={current_k}, QIs left={len(current_qis)}")

    return dropped, current_k, current_qis

In [10]:
dropped_cols, final_k, remaining_qis = greedy_column_suppression_until_k(
    df, identifiers, target_k=5, count_na_as_value=True, verbose=True
)

# Save if you hit k >= 5
if final_k >= 5:
    keep_cols = identifiers + remaining_qis
    cs_df = df[keep_cols].copy()
    cs_df.to_csv(OUT_COLUMN_SUPP, index=False)

Start: k=1, QIs=13
Dropped 'postalCode' (card=18489) -> k=1, QIs left=12
Dropped 'city' (card=13276) -> k=1, QIs left=11
Dropped 'nforum_events' (card=645) -> k=1, QIs left=10
Dropped 'cc_by_ip' (card=218) -> k=1, QIs left=9
Dropped 'YoB' (card=124) -> k=1, QIs left=8
Dropped 'nforum_posts' (card=110) -> k=1, QIs left=7
Dropped 'nforum_comments' (card=104) -> k=1, QIs left=6
Dropped 'nforum_votes' (card=91) -> k=1, QIs left=5
Dropped 'nforum_threads' (card=55) -> k=1, QIs left=4
Dropped 'LoE' (card=12) -> k=1, QIs left=3
Dropped 'nforum_endorsed' (card=11) -> k=1, QIs left=2
Dropped 'nforum_pinned' (card=9) -> k=978, QIs left=1


In [87]:
# ===== configure "high" threshold for forum stats here =====
HIGH_INTERACTION_THRESHOLD = 1  # 1 if value >= 21, else 0


def postal_to_binary(s: pd.Series) -> pd.Series:
    """
    Map postal codes by their first digit:
      - '0'–'4' -> 0
      - '5'–'9' -> 1
      - missing/invalid -> 'Unknown'
    """
    vals = s.astype(str).str.strip()
    out = []
    for v in vals:
        if v == "" or v.lower() in {"nan","none","null","na"}:
            out.append(1); continue
        m = re.search(r"\d", v)
        if not m:
            out.append(1); continue
        d = int(m.group(0))
        out.append(0 if d <= 4 else 1)
    return pd.Series(out, index=s.index, dtype="object")


# Hemisphere lookup: assign countries to N or S hemisphere
_NORTH = {"US","CA","MX","GL","GB","IE","FR","DE","ES","IT","NL","BE","LU","PT","SE","NO","DK","FI","IS",
          "PL","CZ","SK","HU","AT","CH","SI","HR","BA","RS","ME","MK","AL","GR","RO","BG","LT","LV","EE",
          "UA","MD","BY","CN","JP","KR","TW","HK","MO","IN","PK","BD","LK","NP","BT","MV","SG","MY","TH",
          "VN","PH","ID","KH","LA","MM","MN","KZ","UZ","TM","TJ","KG","AE","SA","IR","IQ","IL","JO","LB",
          "SY","TR","YE","OM","QA","KW","BH","AM","AZ","GE","ZA","EG","NG","ET","KE","TZ","DZ","MA","TN",
          "GH","CI","CM","UG","SD","SN","RW","ZM","ZW","MW","MZ","AO","NA","BW","GA","GM","GN","GW","LS",
          "LR","LY","ML","MR","NE","SC","SL","SO","SS","TD","TG","BI","BJ","BF","CD","CG","CV","DJ","ER",
          "GQ","KM","MG","MU","RE","SH","ST","SZ","AU","NZ"}  # very rough cut, adjust as needed

def to_hemisphere(s: pd.Series) -> pd.Series:
    vals = s.astype(str).str.strip()
    out = []
    for v in vals:
        vv = v.upper()
        if vv in _NORTH:
            out.append("North")
        elif vv == "" or vv in {"NAN","NONE","UNKNOWN"}:
            out.append("North")
        else:
            out.append("South")
    return pd.Series(out, index=vals.index, dtype="object")

# Representative "largest" city per hemisphere
_HEMISPHERE_LARGEST_CITY = {
    "North": "Tokyo",     # largest in northern hemisphere
    "South": "São Paulo", # largest in southern hemisphere
}


# ---- helpers ----
def birth_to_pre1990_plus(s: pd.Series) -> pd.Series:
    s_str = s.astype("string").str.strip()
    years_num = pd.to_numeric(s_str, errors="coerce")
    years_dt = pd.to_datetime(s_str, errors="coerce")
    years = years_num.fillna(years_dt.dt.year)

    out = pd.Series("2000+", index=s.index, dtype="object")
    mask_unknown = years.isna()
    mask_pre = (years < 2000).fillna(False)
    out[mask_pre] = "pre-2000"
    out[mask_unknown] = "pre-2000"
    return out

# keep if you still need categorical bins elsewhere
def counts_to_categories(x: pd.Series) -> pd.Series:
    xi = pd.to_numeric(x, errors="coerce")
    bins   = [-np.inf, 0, 5, 20, np.inf]
    labels = ["0", "1-5", "6-20", "21+"]
    return pd.cut(xi.fillna(-np.inf), bins=bins, labels=labels, include_lowest=True).astype(str)

# NEW: binary high/low for interaction-like columns
def counts_to_binary(x: pd.Series, threshold: int = HIGH_INTERACTION_THRESHOLD) -> pd.Series:
    xi = pd.to_numeric(x, errors="coerce").fillna(0)
    return (xi >= threshold).astype(int)

def loe_to_binary(s: pd.Series) -> pd.Series:
    v = s.astype("string").str.strip().str.lower()
    v = v.replace({"": pd.NA, "nan": pd.NA, "none": pd.NA, "null": pd.NA, "unknown": pd.NA})
    def map_loe(t):
        if t is None or t is pd.NA: return "≤Secondary"
        if any(k in t for k in ["less than","primary","elementary","middle","secondary","high school","hs"]):
            return "≤Secondary"
        if "associate" in t: return "Tertiary+"
        if any(k in t for k in ["bachelor","college","undergrad"]): return "Tertiary+"
        if any(k in t for k in ["master","graduate","professional"]): return "Tertiary+"
        if any(k in t for k in ["doctor","doctoral","phd","dphil","md","jd"]): return "Tertiary+"
        return "≤Secondary"
    return v.map(map_loe)


def postal_first_digit_or_00000(s: pd.Series) -> pd.Series:
    vals = s.astype(str)
    out = []
    for v in vals:
        v_strip = v.strip()
        if v_strip == "" or v_strip.lower() in {"nan", "none", "null", "na"}:
            out.append("00000"); continue
        m = re.search(r"\d", v_strip)
        out.append(m.group(0) + "0000" if m else "00000")
    return pd.Series(out, index=s.index, dtype="object")

# ---- single generalization pass ----
def generalize(data: pd.DataFrame) -> pd.DataFrame:
    """
    One-pass generalization:
      - Birth (year or full date) -> {'pre-1990','1990+','Unknown'}
      - Country/region -> continent (NA/EU/AS/AF/OC/SA/Unknown)
      - City -> largest city label for that continent
      - Postal code -> first digit or '00000' (as 'D0000')
      - Level of Education -> binary ('≤Secondary' vs 'Tertiary+')
      - Interaction/counter fields -> BINARY high (>= {thr})=1, else 0
      - Gender -> normalize: any 'o'->'m', NaN->'f'
      - Other columns unchanged
    """
    g = data.copy()
    lower = {c.lower(): c for c in g.columns}

    # Birth → pre-1990 / 1990+
    for key in ("date of birth","dob","birthdate","birth date","year of birth","yob","ear of birth"):
        if key in lower:
            col = lower[key]
            g[col] = birth_to_pre1990_plus(g[col])
            break


    # Country/region → hemisphere
    hemi_col = None
    for key in ("cc_by_ip","country_by_ip","country","region"):
        if key in lower:
            hemi_col = lower[key]
            g[hemi_col] = to_hemisphere(g[hemi_col])
            break

    # City → largest city in that hemisphere
    if "city" in lower:
        city_col = lower["city"]
        if hemi_col is not None:
            g[city_col] = g[hemi_col].map(_HEMISPHERE_LARGEST_CITY).fillna("UnknownCity")
        else:
            g[city_col] = "UnknownCity"

    # Postal code → first digit or 00000
    for key in ("postal_code","postalcode","postal code","zip","zip_code"):
        if key in lower:
            pcol = lower[key]
            g[pcol] = postal_to_binary(g[pcol])
            break

    # Level of Education → binary
    for key in ("level of education","loe","education_level","education level"):
        if key in lower:
            loe_col = lower[key]
            g[loe_col] = loe_to_binary(g[loe_col])
            break

    # Interaction / counter fields → BINARY
    tokens = ("nforum","number of ","events within the forum","interactions",
              "posts","comments","views","votes","clicks","plays","nevents","nplay")
    for c in g.columns:
        name = c.lower()
        if any(t in name for t in tokens):
            try:
                g[c] = counts_to_binary(g[c])  # 1 if >= threshold, else 0
            except Exception:
                pass

    # Gender normalization: any 'o' -> 'm', NaN -> 'f'
    if "gender" in lower:
        col = lower["gender"]
        s = g[col].astype("string").str.strip().str.lower()
        s = s.replace({"": pd.NA, "nan": pd.NA, "none": pd.NA, "null": pd.NA})
        s = s.replace({"o": "m", "male": "m", "m": "m", "female": "f", "f": "f"})
        g[col] = s.fillna("f")

    return g

# (optional) k utility
def k_anonymity_level(df: pd.DataFrame, qis: List[str]) -> int:
    if not qis:
        return len(df)
    sizes = df.groupby(qis, dropna=False).size()
    return int(sizes.min()) if len(sizes) else 0

In [88]:
gdf = generalize(df)
gdf.to_csv(OUT_GEN, index=False)
print("k after full generalization:", k_anonymity_level(gdf, qi_cols))

/var/folders/_4/ykqt5y3x1g9g_q8_17d7jbhm0000gn/T/ipykernel_60157/56986269.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  years_dt = pd.to_datetime(s_str, errors="coerce")


k after full generalization: 1


In [89]:
# Find group sizes for the QI columns
sizes = gdf.groupby(qi_cols, dropna=False).size()

# Mask for groups with size = 1
rare_groups = sizes[sizes == 1]

print("Number of k=1 groups:", len(rare_groups))

# Get the actual records that belong to those groups
violating_records = gdf.merge(
    rare_groups.rename("count"),
    how="inner",
    left_on=qi_cols,
    right_index=True
)

print("Violating records shape:", violating_records.shape)
print(violating_records.head())

Number of k=1 groups: 41
Violating records shape: (41, 16)
                      course_id   user_id cc_by_ip       city postalCode  \
3324    HarvardX/ER22.1x/1T2018  14527936    North      Tokyo          0   
5471    HarvardX/HKS101A/2015T3   3958359    South  São Paulo          0   
5813    HarvardX/HKS101A/2015T3   6494823    North      Tokyo          0   
6107    HarvardX/HKS101A/2015T3   7577470    North      Tokyo          1   
8997  HarvardX/PH201x/2013_SOND   1130567    North      Tokyo          1   

             LoE       YoB gender  nforum_posts  nforum_votes  \
3324  ≤Secondary  pre-2000      m             1             1   
5471  ≤Secondary  pre-2000      m             1             1   
5813  ≤Secondary     2000+      m             1             0   
6107  ≤Secondary  pre-2000      f             1             1   
8997  ≤Secondary  pre-2000      f             1             1   

      nforum_endorsed  nforum_threads  nforum_comments  nforum_pinned  \
3324                

In [60]:
gdf.head()

,course_id,user_id,cc_by_ip,city,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events
0,HarvardX/PH525.1x/1T2018,29940,North,Tokyo,1,≤Secondary,pre-2000,f,0,0,0,0,0,0,0
1,HarvardX/PH525.1x/1T2018,37095,North,Tokyo,1,≤Secondary,pre-2000,m,0,0,0,0,0,0,0
2,HarvardX/PH525.1x/1T2018,45634,South,São Paulo,1,≤Secondary,pre-2000,m,0,0,0,0,0,0,0
3,HarvardX/PH525.1x/1T2018,52234,North,Tokyo,1,≤Secondary,pre-2000,m,0,0,0,0,0,0,0
4,HarvardX/PH525.1x/1T2018,52238,North,Tokyo,1,≤Secondary,pre-2000,f,0,0,0,0,0,0,0


In [91]:
record_supressed_gdf, deleted = record_suppression_k(gdf, qi_cols)
print(deleted)
print(k_anonymity_level(record_supressed_gdf, qi_cols))
print(record_supressed_gdf.shape)

157
5
(199842, 15)


In [67]:
gdf["nforum_posts"].describe()

count    199999.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: nforum_posts, dtype: float64

In [106]:
gdf.head()

,course_id,user_id,cc_by_ip,city,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events
0,HarvardX/PH525.1x/1T2018,29940,NA,Mexico City,70000,≤Secondary,Unknown,f,0,0,0,0,0,0,0
1,HarvardX/PH525.1x/1T2018,37095,AS,Tokyo,00000,≤Secondary,1990+,m,0,0,0,0,0,0,0
2,HarvardX/PH525.1x/1T2018,45634,SA,Sao Paulo,00000,≤Secondary,pre-1990,m,0,0,0,0,0,0,0
3,HarvardX/PH525.1x/1T2018,52234,EU,Moscow,00000,≤Secondary,pre-1990,m,0,0,0,0,0,0,0
4,HarvardX/PH525.1x/1T2018,52238,NA,Mexico City,00000,≤Secondary,Unknown,f,0,0,0,0,0,0,0


In [11]:

# Choose Stage 1 if it helps; otherwise Stage 2
base_gen = g1 if g1_k >= 5 else g2
base_qis = [c for c in base_gen.columns if c not in identifiers]
combo_df, combo_deleted = record_suppression_k(base_gen, base_qis, k=5)
combo_k = k_anonymity_level(combo_df, base_qis)
combo_df.to_csv(OUT_COMBINATION, index=False)

print({
    "rows_source": len(df),
    "deleted_rows": combo_deleted,
    "rows_kept": len(combo_df),
    "k_after": combo_k,
    "file": OUT_COMBINATION
})

{'rows_source': 199999, 'deleted_rows': 68207, 'rows_kept': 131792, 'k_after': 5, 'file': '/Users/andrewrodriguez/Desktop/compsci1050/anonymity_and_ethics/de-identified_data/combination.csv'}
